# Workflow Examples

This chapter demonstrates three common workflow patterns using the nodes from the previous chapter. Import the pre-built workflows, run them, and observe how the nodes work together.

**Workflow files:** The JSON files are in `courses/n8n_no_code/workflows/`

**Note on credentials:** These workflows require API credentials for an AI provider (OpenRouter, OpenAI, or Google). If you do not have credentials set up, you can still explore the workflow structure and understand what each node would do.

---

## How to Import and Run a Workflow

**Import:**
1. In n8n, click **Workflows** in the left sidebar
2. Click **Add Workflow**
3. Click the **three-dot menu (⋮)** in the top-right
4. Select **Import from File**
5. Choose the `.json` file
6. Click **Save**

**Run:**
1. Open the workflow in the editor
2. Make sure credentials are set up (Settings → Credentials)
3. Click **Execute Workflow** in the top toolbar
4. Click any node to see its output in the right panel

---

## Pattern 1: Prompt Chaining

**File:** `01_prompt_chaining.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/workflows/01_prompt_chaining.json
> ```
> 
> **Download:** [01_prompt_chaining.json](https://github.com/ezponda/ai-agents-course/raw/main/courses/n8n_no_code/workflows/01_prompt_chaining.json)

### Nodes in This Pattern

This pattern uses nodes you already know from Quick Start:

| Node | What it does |
|------|--------------|
| **Manual Trigger** | Starts the workflow when you click "Test workflow" |
| **Edit Fields (Set)** | Creates or transforms data fields |
| **Basic LLM Chain** | Sends a prompt to an AI model and returns the response |

The difference here: we chain **multiple** Basic LLM Chains in sequence, where each uses the previous one's output.

### What Problem This Solves

Breaking a complex writing task into smaller steps improves quality. Instead of asking an LLM to "write a memo" in one shot, you ask it to: (1) create an outline, (2) improve the outline, (3) write the final draft. Each step builds on the previous result.

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Run: Prompt Chaining** | Manual Trigger | Starts the workflow |
| **Input — Writing Brief** | Set | Creates fields: `topic`, `audience`, `constraints` |
| **Step 1 — Create Outline** | Basic LLM Chain | Generates an outline → outputs `text` |
| **Store Outline** | Set | Saves `{{ $json.text }}` as `outline` |
| **Step 2 — Improve Outline** | Basic LLM Chain | Refines the outline using `{{ $json.outline }}` → outputs `text` |
| **Store Improved Outline** | Set | Saves `{{ $json.text }}` as `improved_outline` |
| **Step 3 — Draft Memo** | Basic LLM Chain | Writes final memo using `{{ $json.improved_outline }}` → outputs `text` |
| **Output — Final Memo** | Set | Saves `{{ $json.text }}` as `final_memo` (uses `keepOnlySet` to remove other fields) |

**Sub-node:** One `OpenRouter Chat Model` is shared by all three LLM Chain nodes (connected via dotted lines).

### Prompts Used

Each step uses a different **role** and **rules**. This is why chaining works better than one big prompt.

**Step 1 — Create Outline:**
```
System: You are a writing assistant.
Create a clear outline for the memo.

Rules:
- 5–7 sections
- Include where the benefits bullets and risks bullet will go
- Output ONLY the outline
```

**Step 2 — Improve Outline:**
```
System: You are a strict editor.
Improve the outline using this checklist:
- Logical flow
- Clear headings
- Covers constraints (benefits + risks)
- Non-technical wording

Output ONLY the improved outline (no commentary).
```

**Step 3 — Draft Memo:**
```
System: You are a business writer.
Write the memo based on the outline.

Rules:
- Follow the outline order
- Keep under 400 words
- Use clear, non-technical language
- Include exactly: 3 benefit bullets + 1 risk bullet

Output ONLY the memo text.
```

**Why this works:** Each step has a focused role (assistant → editor → writer), specific constraints, and ends with "Output ONLY..." to prevent extra commentary.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Writing Brief: { topic, audience, constraints }
    ↓
Step 1 LLM: { text: "1. Introduction..." }
    ↓
Store Outline: { outline: "1. Introduction..." }
    ↓
Step 2 LLM: { text: "1. Executive Summary..." }
    ↓
Store Improved: { improved_outline: "1. Executive Summary..." }
    ↓
Step 3 LLM: { text: "MEMO: Why We Should..." }
    ↓
Final Output: { final_memo: "MEMO: Why We Should..." }
```

### What to Observe

1. Click **Input — Writing Brief** → see the starting data
2. Click **Step 1 — Create Outline** → see the LLM's outline in `text`
3. Click **Store Outline** → see the same content now saved as `outline`
4. Follow this pattern through each step to see how data transforms

---

## Pattern 2: Routing

**File:** `02_routing.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/workflows/02_routing.json
> ```
> 
> **Download:** [02_routing.json](https://github.com/ezponda/ai-agents-course/raw/main/courses/n8n_no_code/workflows/02_routing.json)

### Meet the Node: Switch

This pattern introduces a new node: **Switch**.

| Property | Description |
|----------|-------------|
| **Purpose** | Routes data to different branches based on conditions |
| **How it works** | Checks a value and sends data to the matching output |
| **Multiple outputs** | Each condition creates a separate output branch (solid line) |

**Configuration:**
1. **Mode:** Choose "Rules" for condition-based routing
2. **Data Type:** Usually "String" when routing on text values
3. **Value 1:** The field to check, e.g., `{{ $json.route }}`
4. **Operation:** Usually "Equals" for exact matching
5. **Value 2:** The value to match, e.g., `refund`

Add more rules to create more branches. The Switch sends data only to the branch where the condition matches.

**See Node Toolbox for full documentation.**

### What Problem This Solves

Different inputs need different handling. A support ticket about refunds should go to a refund specialist prompt, while an order status question goes elsewhere. Routing uses an LLM to classify the input, then a Switch node sends it down the right path.

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Run: Ticket Routing** | Manual Trigger | Starts the workflow |
| **Input — Support Ticket** | Set | Creates fields: `ticket_subject`, `ticket_body` |
| **Router — Choose Route** | Basic LLM Chain | Classifies ticket → outputs `text` (e.g., `refund`) |
| **Store Route** | Set | Saves `{{ $json.text.trim() }}` as `route` |
| **Switch — Route** | Switch | Checks `{{ $json.route }}` and sends to one branch |
| **Refund Specialist — Draft Reply** | Basic LLM Chain | Writes refund-specific reply → outputs `text` |
| **Order Status Specialist — Draft Reply** | Basic LLM Chain | Writes order status reply → outputs `text` |
| **Support Specialist — Draft Reply** | Basic LLM Chain | Writes general support reply → outputs `text` |
| **Output — Refund Reply** | Set | Saves `{{ $json.text }}` as `reply`, adds `route: "refund"` |
| **Output — Order Status Reply** | Set | Saves `{{ $json.text }}` as `reply`, adds `route: "order_status"` |
| **Output — Support Reply** | Set | Saves `{{ $json.text }}` as `reply`, adds `route: "support"` |

### Prompts Used

**Router — Choose Route (the classifier):**
```
System: You route customer support tickets.

Choose exactly ONE route label, lowercase, no punctuation:
- refund
- order_status
- support

Return ONLY the label.
```

**Why this works:** The prompt forces clean output (lowercase, no punctuation, ONLY the label) so the Switch node can match exactly.

**Refund Specialist:**
```
System: You are a customer support specialist for refunds.
Write a short, professional reply.

Rules:
- Acknowledge the issue
- Ask for any missing info (only if needed)
- Explain next steps and expected timeline
- Keep it under 120 words
```

**Order Status Specialist:**
```
System: You are a customer support specialist for order status.
Write a short, professional reply.

Rules:
- Confirm you are checking the order
- Ask for order number if missing
- Provide what you can and what you still need
- Keep it under 120 words
```

**Support Specialist:**
```
System: You are a general customer support specialist.
Write a short, professional reply.

Rules:
- Clarify the problem
- Ask 1–2 targeted questions (only if needed)
- Offer a next step
- Keep it under 120 words
```

### Key Expression: Accessing Earlier Node Data

The specialist nodes need the original ticket data, but they come after the Switch. They use this expression pattern:

```
{{ $node['Input — Support Ticket'].json.ticket_subject }}
{{ $node['Input — Support Ticket'].json.ticket_body }}
```

This accesses data from a specific earlier node by name. Useful when you need to "reach back" past intermediate nodes.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Support Ticket: { ticket_subject, ticket_body }
    ↓
Router LLM: { text: "refund" }
    ↓
Store Route: { route: "refund", ticket_subject, ticket_body }
    ↓
Switch: routes to ONE branch based on {{ $json.route }}
    ↓
[Refund Specialist]: { text: "Dear Jamie, I apologize..." }
    ↓
Output — Refund Reply: { reply: "Dear Jamie, I apologize...", route: "refund" }
```

### What to Observe

1. Click **Router — Choose Route** → see the classification in `text` (e.g., `refund`)
2. Click **Store Route** → see `route` field added
3. Click **Switch — Route** → only ONE output branch has data
4. Click the active specialist node → see the tailored reply

---

## Pattern 3: Parallelization

**File:** `03_parallelization.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/workflows/03_parallelization.json
> ```
> 
> **Download:** [03_parallelization.json](https://github.com/ezponda/ai-agents-course/raw/main/courses/n8n_no_code/workflows/03_parallelization.json)

### Meet the Node: Merge

This pattern introduces a new node: **Merge**.

| Property | Description |
|----------|-------------|
| **Purpose** | Combines data from multiple branches into one |
| **Multiple inputs** | Receives data from 2+ parallel branches |
| **Output** | One combined item (or list) with all fields |

**Key Modes:**
| Mode | What it does |
|------|--------------|
| **Combine by Position** | Pairs items by index (1st + 1st, 2nd + 2nd). Best when each branch outputs one item. |
| **Combine by Fields** | Matches items by a common field value |
| **Append** | Puts all items into one list |

In this workflow, we use **Combine by Position** because each branch outputs exactly one item.

**See Node Toolbox for full documentation.**

### What Problem This Solves

Some tasks have independent parts that can run at the same time. Analyzing a customer email for facts, sentiment, and drafting a reply are independent—they don't need each other's results. Running them in parallel is faster, and you can combine the results at the end for a final, informed response.

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Run: Parallelization** | Manual Trigger | Starts the workflow |
| **Input — Customer Email** | Set | Creates fields: `email_subject`, `email_body` |
| **Branch A — Extract Facts** | Basic LLM Chain | Extracts facts as JSON → outputs `text` |
| **Store Facts** | Set | Saves `{{ $json.text }}` as `facts_json` |
| **Branch B — Sentiment & Urgency** | Basic LLM Chain | Analyzes sentiment → outputs `text` |
| **Store Sentiment** | Set | Saves `{{ $json.text }}` as `sentiment_json` |
| **Branch C — Draft Reply** | Basic LLM Chain | Drafts initial reply → outputs `text` |
| **Store Draft Reply** | Set | Saves `{{ $json.text }}` as `draft_reply` |
| **Merge A+B** | Merge | Combines `facts_json` + `sentiment_json` (mode: Combine by Position) |
| **Merge (A+B)+C** | Merge | Adds `draft_reply` to the combined data |
| **Finalize — One Improved Reply** | Basic LLM Chain | Uses all three fields → outputs `text` |
| **Output — Final Reply** | Set | Saves `{{ $json.text }}` as `final_reply` |

### Prompts Used

**Branch A — Extract Facts:**
```
System: Extract key facts from the email.
Return STRICT JSON with keys:
customer_name, issue, deadline, requested_action, missing_info (array).
Return JSON only.
```

**Branch B — Sentiment & Urgency:**
```
System: Classify sentiment and urgency.
Return STRICT JSON with keys:
sentiment (positive|neutral|negative), urgency (low|medium|high), risk_flags (array).
Return JSON only.
```

**Branch C — Draft Reply:**
```
System: Draft a helpful customer support email reply.
Rules:
- Friendly and concise
- Ask for missing info only if needed
- Offer 1–2 concrete next steps
- Under 140 words

Output ONLY the reply text.
```

**Finalize — One Improved Reply:**
```
System: You are a senior support agent.
You will receive:
- facts_json (extracted facts)
- sentiment_json (sentiment & urgency)
- draft_reply (initial draft)

Task:
1) Improve the draft to match urgency and include any critical missing info questions.
2) Keep it under 160 words.
3) Output ONLY the final reply text.
```

### Example JSON Outputs

**Branch A (facts_json):**
```json
{
  "customer_name": "Sam",
  "issue": "Can't log in after password reset, invalid token error",
  "deadline": "end of day",
  "requested_action": "Help accessing invoices",
  "missing_info": []
}
```

**Branch B (sentiment_json):**
```json
{
  "sentiment": "negative",
  "urgency": "high",
  "risk_flags": ["access issue", "time-sensitive", "repeated attempts"]
}
```

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Customer Email: { email_subject, email_body }
    ↓ ↓ ↓ (splits into 3 parallel branches)
    
Branch A: { text: '{"customer_name":"Sam",...}' }
    ↓
Store Facts: { facts_json: '{"customer_name":"Sam",...}' }

Branch B: { text: '{"sentiment":"negative","urgency":"high",...}' }
    ↓
Store Sentiment: { sentiment_json: '{"sentiment":"negative",...}' }

Branch C: { text: "Hi Sam, I understand..." }
    ↓
Store Draft Reply: { draft_reply: "Hi Sam, I understand..." }
    
    ↓ (merge all three)
After Merge: { facts_json, sentiment_json, draft_reply }
    ↓
Finalize LLM: { text: "Dear Sam, I sincerely apologize..." }
    ↓
Final Output: { final_reply: "Dear Sam, I sincerely apologize..." }
```

### What to Observe

1. Click **Input — Customer Email** → see the starting data
2. Click each **Branch** node → see different analyses running independently
3. Click **Merge (A+B)+C** → switch to JSON view to see all three fields combined
4. Click **Finalize** → see how the final LLM uses `{{ $json.facts_json }}`, `{{ $json.sentiment_json }}`, and `{{ $json.draft_reply }}`

---

## AI Agent Node (Minimal Intro)

The **AI Agent** node is a workflow step that can **decide what to do next** instead of following a single fixed prompt.

- **Tools:** optional actions the agent can call (e.g., calculator, HTTP request, search). Think: "abilities".
- **Memory:** optional context the agent can carry across steps so it stays consistent.
- **Loop (plan → act → check):** the agent can iterate: decide a plan, use a tool (act), check results, and stop when done.

**Tip:** Start simple—use **one tool** (or none) and keep runs in **manual test mode** to avoid accidental costs.

See the **AI Agent Examples** chapter for a hands-on walkthrough.

---

## Pattern Summary

| Pattern | When to use | Key nodes |
|---------|-------------|------------|
| **Prompt Chaining** | Complex tasks that benefit from step-by-step refinement | Multiple LLM Chains in sequence |
| **Routing** | Different handling based on input type | LLM (classifier) + Switch |
| **Parallelization** | Independent analyses that can run simultaneously | Multiple branches + Merge |

---

## Tips for Building Your Own Workflows

1. **Start simple.** Build one node at a time and test frequently.

2. **Use descriptive node names.** "Step 1: Create Outline" is better than "LLM Chain."

3. **Pin data often.** After any successful LLM call, pin the result before working on the next node.

4. **Check the Output panel.** Switch between Table and JSON views to understand your data.

5. **Use Set nodes as checkpoints.** Save intermediate results with clear field names.

6. **Test routing with different inputs.** Make sure all branches work, not just the happy path.